In [18]:
# app.py
import os
import tempfile
from datetime import datetime

import joblib
import librosa
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import sounddevice as sd
import soundfile as sf
import streamlit as st

# ----------------------------
# CONFIG
# ----------------------------
MODEL_FILE = "generalized_emotion_model.joblib"
SESSION_LOG = "session_log.csv"
RECORD_SECONDS_DEFAULT = 3
SAMPLE_RATE = 22050  # Must match training SR
N_MFCC = 40  # mfcc count used in training

st.set_page_config(page_title="🎙️ Live Voice Emotion (Record/Upload)", layout="centered")

# ----------------------------
# Load model bundle
# ----------------------------
@st.cache_resource
def load_model(bundle_path=MODEL_FILE):
    if not os.path.exists(bundle_path):
        return None, None
    bundle = joblib.load(bundle_path)
    model = bundle.get("model") if isinstance(bundle, dict) else bundle
    label_encoder = bundle.get("label_encoder") if isinstance(bundle, dict) else None
    return model, label_encoder

model, label_encoder = load_model()
if model is None or label_encoder is None:
    st.error(f"Model bundle not found or invalid. Expected '{MODEL_FILE}' containing {{'model','label_encoder'}}.")
    st.stop()

# ----------------------------
# Emotion color gradients (card-style)
# ----------------------------
EMO_COLORS = {
    "calm": ("#a8dadc", "#457b9d"),
    "happy": ("#fff3b0", "#ffb703"),
    "sad": ("#d8c1f0", "#6a4c93"),
    "angry": ("#ffd6d6", "#ff4d4d"),
    "fearful": ("#ffd8b1", "#f07066"),
    "disgust": ("#c7f9cc", "#2b9348"),
    "surprised": ("#ffd6ff", "#ff61a6"),
    "neutral": ("#e9ecef", "#6c757d")
}

# Fallback color
DEFAULT_GRADIENT = ("#f6f6f6", "#d1d1d1")

# ----------------------------
# Utility: feature extraction (MFCC + delta + delta-delta -> 120 features)
# ----------------------------
def extract_features(file_path, sr=SAMPLE_RATE, n_mfcc=N_MFCC):
    y, sr = librosa.load(file_path, sr=sr, mono=True)
    y, _ = librosa.effects.trim(y)

    # Core MFCCs
    mfcc = librosa.feature.mfcc(y=y, sr=sr, n_mfcc=n_mfcc)
    mfcc_mean = np.mean(mfcc, axis=1)

    # Delta and Delta-Delta
    delta_mfcc = librosa.feature.delta(mfcc)
    delta2_mfcc = librosa.feature.delta(mfcc, order=2)
    delta_mean = np.mean(delta_mfcc, axis=1)
    delta2_mean = np.mean(delta2_mfcc, axis=1)

    features = np.hstack([mfcc_mean, delta_mean, delta2_mean])
    return features.reshape(1, -1)

# ----------------------------
# Recording helper
# ----------------------------
def record_audio(duration=RECORD_SECONDS_DEFAULT, sr=SAMPLE_RATE):
    st.info(f"Recording for {duration} seconds... (make sure your microphone is enabled)")
    try:
        recording = sd.rec(int(duration * sr), samplerate=sr, channels=1, dtype="float32")
        sd.wait()
    except Exception as e:
        st.error("Recording failed: " + str(e))
        return None

    tmp = tempfile.NamedTemporaryFile(suffix=".wav", delete=False)
    sf.write(tmp.name, recording.flatten(), sr)
    return tmp.name

# ----------------------------
# Session-state init (history)
# ----------------------------
if "history_df" not in st.session_state:
    if os.path.exists(SESSION_LOG):
        try:
            st.session_state["history_df"] = pd.read_csv(SESSION_LOG)
        except Exception:
            st.session_state["history_df"] = pd.DataFrame()
    else:
        st.session_state["history_df"] = pd.DataFrame()

if "current_session" not in st.session_state:
    st.session_state["current_session"] = []

# ----------------------------
# Small helper: build gradient card HTML
# ----------------------------
def emotion_card_html(emotion, confidence, emoji="🎯"):
    emo_lower = str(emotion).lower()
    grad = EMO_COLORS.get(emo_lower, DEFAULT_GRADIENT)
    color_from, color_to = grad
    conf_pct = f"{confidence*100:.1f}%"
    # Icon map (simple)
    ICONS = {
        "calm": "🌊",
        "happy": "😄",
        "sad": "😢",
        "angry": "😠",
        "fearful": "😨",
        "disgust": "🤢",
        "surprised": "😲",
        "neutral": "😐"
    }
    icon = ICONS.get(emo_lower, "🎯")
    html = f"""
    <div style="padding:16px;border-radius:14px;
                background: linear-gradient(135deg, {color_from}, {color_to});
                box-shadow: 0 6px 18px rgba(0,0,0,0.12);
                color:#0b0b0b;">
      <div style="display:flex; align-items:center; gap:12px;">
        <div style="font-size:44px;">{icon}</div>
        <div>
          <div style="font-size:20px; font-weight:700; letter-spacing:0.5px;">{emotion.upper()}</div>
          <div style="margin-top:6px; font-size:14px; opacity:0.95;">Confidence: <strong>{conf_pct}</strong></div>
        </div>
      </div>
    </div>
    """
    return html

# ----------------------------
# Prediction + logging logic (shared for record/upload)
# ----------------------------
def predict_and_log(audio_path):
    try:
        feats = extract_features(audio_path)
        probs = model.predict_proba(feats)[0]  # probabilities aligned with model.classes_
        class_indices = model.classes_
        # Map encoded class indices to string labels using label_encoder
        labels = label_encoder.inverse_transform(class_indices.astype(int))
        # Build prob dataframe in the correct order
        prob_df = pd.DataFrame({"emotion": labels, "probability": probs})
        # Top prediction
        top_idx = int(np.argmax(probs))
        top_label = labels[top_idx]
        top_conf = float(probs[top_idx])

        # Show card (HTML)
        card_html = emotion_card_html(top_label, top_conf)
        st.markdown(card_html, unsafe_allow_html=True)

        # Show probability bar chart (matplotlib)
        prob_df_sorted = prob_df.sort_values("probability", ascending=False).reset_index(drop=True)
        fig, ax = plt.subplots(figsize=(6, 3.5))
        ax.barh(prob_df_sorted["emotion"][::-1], prob_df_sorted["probability"][::-1])
        ax.set_xlim(0, 1)
        ax.set_xlabel("Probability")
        ax.set_title("Emotion probabilities")
        plt.tight_layout()
        st.pyplot(fig)

        # Also show a neat table
        prob_table = prob_df_sorted.copy()
        prob_table["probability"] = (prob_table["probability"] * 100).map(lambda x: f"{x:.1f}%")
        st.table(prob_table)

        # Log row (timestamp, predicted, prob_<label>...)
        timestamp = datetime.now().isoformat()
        log_row = {"timestamp": timestamp, "predicted": top_label}
        for i, lab in enumerate(labels):
            log_row[f"prob_{lab}"] = float(probs[i])

        # Append to CSV
        if os.path.exists(SESSION_LOG):
            try:
                existing = pd.read_csv(SESSION_LOG)
                existing = pd.concat([existing, pd.DataFrame([log_row])], ignore_index=True)
                existing.to_csv(SESSION_LOG, index=False)
            except Exception:
                pd.DataFrame([log_row]).to_csv(SESSION_LOG, index=False)
        else:
            pd.DataFrame([log_row]).to_csv(SESSION_LOG, index=False)

        # Update session state (history df and current in-memory list)
        if "history_df" not in st.session_state:
            st.session_state["history_df"] = pd.DataFrame([log_row])
        else:
            st.session_state["history_df"] = pd.concat([st.session_state["history_df"], pd.DataFrame([log_row])], ignore_index=True)
        st.session_state["current_session"].append(log_row)

    except Exception as e:
        st.error("Prediction failed: " + str(e))
    finally:
        # cleanup
        try:
            os.remove(audio_path)
        except Exception:
            pass

# ----------------------------
# UI: Title + description
# ----------------------------
st.title("🎙️ Human Emotion Detection — Record or Upload")
st.markdown(
    "Record a short audio clip or upload a WAV file. The app predicts emotion, shows a colored card with confidence, "
    "probabilities, logs the session and updates the trend graph instantly."
)

# ----------------------------
# Input area: Record or Upload
# ----------------------------
st.subheader("Input Audio")
col1, col2 = st.columns(2)
input_audio_path = None

with col1:
    duration = st.number_input("Record duration (seconds)", min_value=1, max_value=10, value=RECORD_SECONDS_DEFAULT, step=1)
    if st.button("🔴 Record"):
        audio_path = record_audio(duration=duration, sr=SAMPLE_RATE)
        if audio_path:
            st.success("Recording saved.")
            st.audio(audio_path)
            input_audio_path = audio_path

with col2:
    uploaded_file = st.file_uploader("Or upload a .wav file", type=["wav"])
    if uploaded_file is not None:
        tmp = tempfile.NamedTemporaryFile(suffix=".wav", delete=False)
        tmp.write(uploaded_file.read())
        tmp.flush()
        st.success("File uploaded successfully.")
        st.audio(tmp.name)
        input_audio_path = tmp.name

# Run prediction if input is provided
if input_audio_path:
    predict_and_log(input_audio_path)

# ----------------------------
# Trend graph (dynamic, uses session_state history_df)
# ----------------------------
st.markdown("---")
st.subheader("📈 Emotion trend (session history)")

def plot_trend_from_df(df, title="Emotion counts (recent)"):
    if df is None or df.empty:
        st.info("No session history to plot yet. Record or upload some samples.")
        return

    df = df.copy()
    if "predicted" not in df.columns:
        st.info("Session log has no 'predicted' column yet.")
        return
    df["predicted"] = df["predicted"].astype(str)

    N = 50
    last = df.tail(N).reset_index(drop=True)

    emotions = list(label_encoder.classes_)
    counts_over_time = pd.DataFrame(0, index=range(len(last)), columns=emotions)

    for i, emo in enumerate(last["predicted"]):
        if emo in emotions:
            counts_over_time.loc[i, emo] = 1

    cum_counts = counts_over_time.cumsum()

    fig, ax = plt.subplots(figsize=(9, 4.5))
    for emo in emotions:
        ax.plot(cum_counts.index, cum_counts[emo], label=emo, marker="o")
    ax.set_xlabel("Recent session index (most recent on right)")
    ax.set_ylabel("Cumulative count")
    ax.set_title(title)
    ax.legend(loc="upper left", bbox_to_anchor=(1.02, 1.0))
    plt.tight_layout()
    st.pyplot(fig)

plot_trend_from_df(st.session_state.get("history_df", pd.DataFrame()), title="Cumulative emotion counts (last 50 sessions)")

# ----------------------------
# Recent sessions table & controls
# ----------------------------
st.markdown("---")
st.subheader("Recent sessions")

if st.session_state.get("history_df", pd.DataFrame()).empty:
    st.info("No sessions logged yet.")
else:
    recent = st.session_state["history_df"].copy()
    # show latest first
    recent = recent.sort_values("timestamp", ascending=False).reset_index(drop=True)
    st.dataframe(recent.head(10))

col_clear, col_export = st.columns(2)
with col_clear:
    if st.button("Clear history (delete CSV)"):
        st.session_state["history_df"] = pd.DataFrame()
        try:
            if os.path.exists(SESSION_LOG):
                os.remove(SESSION_LOG)
            st.success("Session history cleared.")
        except Exception as e:
            st.error("Failed to clear session file: " + str(e))
with col_export:
    if st.button("Export history CSV"):
        if not st.session_state.get("history_df", pd.DataFrame()).empty:
            tmp = tempfile.NamedTemporaryFile(delete=False, suffix=".csv")
            st.session_state["history_df"].to_csv(tmp.name, index=False)
            with open(tmp.name, "rb") as f:
                st.download_button("Download CSV", f, file_name="session_log.csv")
        else:
            st.info("No data to export.")

# ----------------------------
# Notes & tips
# ----------------------------
st.markdown(
    """
---
**Notes & tips**
- The model expects audio sampled (or resampled) at 22050 Hz (librosa will resample automatically).
- Recording uses the `sounddevice` library (works when running locally). For browser-based recording you can use `streamlit-webrtc`.
- If you retrain the model to include more features, ensure `extract_features` here matches the training features exactly.
- Requirements (suggested):
    pip install streamlit sounddevice soundfile librosa numpy pandas scikit-learn joblib matplotlib
- Run locally:
    streamlit run app.py
"""
)


🎧 Found 2880 audio files.
Processed 200 files...
Processed 400 files...
Processed 600 files...
Processed 800 files...
Processed 1000 files...
Processed 1200 files...
Processed 1400 files...
Processed 1600 files...
Processed 1800 files...
Processed 2000 files...
Processed 2200 files...
Processed 2400 files...
Processed 2600 files...
Processed 2800 files...
✅ Saved features to features_mfcc_delta.csv
⚠️ Skipped 0 files due to errors or unknown labels.
🎯 Original class distribution:
 label
calm         384
happy        384
sad          384
angry        384
fearful      384
disgust      384
surprised    384
neutral      192
Name: count, dtype: int64
🎯 Filtered class distribution:
 label
calm         384
happy        384
sad          384
angry        384
fearful      384
disgust      384
surprised    384
neutral      192
Name: count, dtype: int64
🚀 Training generalized SVM model...
✅ Accuracy: 0.898
              precision    recall  f1-score   support

       angry       0.90      0.98    